# Import a TensorFlow model into IBM Watson Machine Learning

Importing a model into Watson Machine Learning means to store a trained model in your Watson Machine Learning repository and then deploy the stored model.  This notebook demonstrates importing a TensorFlow model.

See also: <a href="https://dataplatform.cloud.ibm.com/docs/content/analyze-data/ml-import-tensorflow.html" target="_blank" rel="noopener noreferrer">Importing a TensorFlow model</a>

This notebook runs on Python 3.5.


### Notebook sections

[Step 0: Build, train, and save a model](#step0)

[Step 1: Store the model in your Watson Machine Learning repository](#step1)

[Step 2: Deploy the stored modelin your Watson Machine Learning service](#step2)

## <a id="step0"></a> Step 0: Build, train, and save a model

**About the sample model**

The sample model built here classifies text messages from fictional customers into two categories:
- "social" : The message might just be social and friendly, or the message lacks enough text to perform in-depth analysis
- "problem or question" : The message describes a prob1em or asks a questions

Classifying messages this way is useful for multiple purposes:
- Automating responses (eg. respond to social messages with a generial greeting, prompting the user to type their question or problem if they have one)
- Cleaning out social message for post-hoc analysis


The data used to train the model is the "GoSales.csv" training data in the IBM Watson Studio community: <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/015ddef6a868441188268a123404f744" target="_blank" rel="noopener noreferrer">Customer messages sample data</a>.

### Get and prepare training data

In [1]:
!pip install wget # Needed to download sample training data

You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# Download sample training data to the notebook working directory and read it into a Pandas DataFrame
import wget
training_data_url = 'https://dataplatform.cloud.ibm.com/data/exchange-api/v1/entries/015ddef6a868441188268a123404f744/data?accessKey=c8d0403d844a82df9ecd264df02f2b07'
filename = wget.download( training_data_url )
print( filename )

sample-customer-messages.csv


In [3]:
# Read sample data into Pandas DataFrame
import pandas as pd
df = pd.read_csv( filename, names=[ "message", "class_name" ] )
df[6:9]

,message,class_name
6,Greetings :),hi
7,hai how can i do analyze with csv file is ther...,question
8,Having issues setup WML service,problem


In [4]:
# Split the data into training and test sets
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
from sklearn.model_selection import train_test_split
train, test = train_test_split( df, test_size = 0.1 )

In [5]:
# Create a vocabulary containing the words in the training data
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit( train["message"] )
vocab_size = len( vectorizer.get_feature_names() )

In [6]:
print( "vocab_size: "+ str( vocab_size) )
print( "\n" + str( vectorizer.get_feature_names()[0:10] ) + " ..." )

vocab_size: 276

['5gb', 'able', 'access', 'account', 'accout', 'acess', 'active', 'add', 'am', 'an'] ...


In [7]:
# Convert the messages to vectors
X_train = vectorizer.transform( train["message"] ).toarray()
X_test  = vectorizer.transform( test["message" ] ).toarray()

In [8]:
test

,message,class_name
58,Hi I would like to signup for a trail but I am...,problem
2,Good morning,hi
30,hey there,hi
88,I get error like Exceeded Services Limit,problem
71,Hi there can i recover a deleted notebook?,question
46,Hi I am trying to Import Github Code To Create...,problem
28,Hello... i created a Cloudent NoSQL DB and loa...,question
77,How do I add collaborators to my project please?,question
1,Good evening,hi
63,hi is there link on how to deploy a data analy...,question


In [9]:
print( X_test[0:1,:] )
print( X_test[1:2,:] )
print( X_test[2:3,:] )

[[0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [10]:
# Convert the labels to binary labels
import scipy
import numpy as np
y_train = np.array( [ [ 1, 0 ] if class_name == "hi" else [ 0, 1 ] for class_name in train[ "class_name" ] ] )
y_test  = np.array( [ [ 1, 0 ] if class_name == "hi" else [ 0, 1 ] for class_name in test[ "class_name" ] ] )
y_test

array([[0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1]])

### Build a neural network

In [11]:
import tensorflow as tf

tf.reset_default_graph()

num_inputs         = vocab_size
num_layer1_nodes   = 50
num_output_classes = 2
learning_rate      = 0.01

#Input: X, labels: y
X = tf.placeholder( tf.float32, shape = ( None, num_inputs ) )
y = tf.placeholder( tf.float32, shape = ( None, num_output_classes ) )

# Layer 1
w1 = tf.Variable( tf.truncated_normal( shape=[ num_inputs, num_layer1_nodes ] ) )
b1 = tf.Variable( tf.zeros( shape=[ num_layer1_nodes ] ) )
layer1_output =  tf.nn.relu( tf.matmul( X, w1 ) +  b1 ) 

# Output
w2 = tf.Variable( tf.truncated_normal( shape=[ num_layer1_nodes, num_output_classes ] ) )
b2 = tf.Variable( tf.zeros( shape=[ num_output_classes ] ) )
output = tf.nn.softmax( tf.matmul( layer1_output, w2 ) + b2 )

loss = tf.losses.softmax_cross_entropy( y, output )

optimizer = tf.train.AdamOptimizer( learning_rate ).minimize( loss )

expected_result = tf.argmax( y,      axis = 1 )
classification  = tf.argmax( output, axis = 1 )
accuracy = tf.reduce_mean( tf.cast( tf.equal( expected_result, classification ), tf.float32 ) )

/Users/jinsalex/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



### Train and evaluate model

In [12]:
num_epochs = 40

session = tf.Session()

session.run( tf.global_variables_initializer() )

accuracy_train = []
accuracy_test  = []

for epoch in range( num_epochs ):
    
    session.run( optimizer, feed_dict={ X : X_train, y : y_train } )
    
    accuracy_train.append( session.run( accuracy, feed_dict = { X : X_train, y : y_train } ) )
    accuracy_test.append(  session.run( accuracy, feed_dict = { X : X_test, y  : y_test  } ) )

In [13]:
import matplotlib.pyplot as plt
plt.style.use( "seaborn-white" )

plt.title(  "Accuracy", fontsize = 18)

x = range( 1, num_epochs + 1 )
plt.plot( x, accuracy_train, label = "Training" )
plt.plot( x, accuracy_test,  label = "Test" )

legend = plt.legend( loc="upper left" )

In [14]:
test

,message,class_name
58,Hi I would like to signup for a trail but I am...,problem
2,Good morning,hi
30,hey there,hi
88,I get error like Exceeded Services Limit,problem
71,Hi there can i recover a deleted notebook?,question
46,Hi I am trying to Import Github Code To Create...,problem
28,Hello... i created a Cloudent NoSQL DB and loa...,question
77,How do I add collaborators to my project please?,question
1,Good evening,hi
63,hi is there link on how to deploy a data analy...,question


In [15]:
test_results = session.run( classification, feed_dict = { X : X_test } )
print( test_results )

[1 0 0 0 0 1 1 1 0 1]


### Save the model
You can import your TensorFlow model into Watson Machine Learning in two formats:
- Model saved in a directory using <a href="https://www.tensorflow.org/guide/saved_model#save_and_restore_models" target="_blank" rel="noopener noreferrer">saved_model</a>
- Model saved in a tar.gz file

In this section of the notebook, the model is saved in a dictionary, and the ditionary is saved in a tar.gz file to demonstrate both options.

In [16]:
classification_inputs         = tf.saved_model.utils.build_tensor_info( X )
classification_output_classes = tf.saved_model.utils.build_tensor_info( classification )

classification_signature = ( tf.saved_model.signature_def_utils.build_signature_def (
    inputs  = { tf.saved_model.signature_constants.CLASSIFY_INPUTS         : classification_inputs },
    outputs = { tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES : classification_output_classes },
    method_name=tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME ) )

#print( classification_signature )

In [17]:
# Save the model in a directory
builder = tf.saved_model.builder.SavedModelBuilder( "message-classification-model-dir" )

builder.add_meta_graph_and_variables(
      session, [ tf.saved_model.tag_constants.SERVING ],
      signature_def_map={ "classify_message" : classification_signature, },
      main_op=tf.tables_initializer() )

builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'message-classification-model-dir/saved_model.pb'


b'message-classification-model-dir/saved_model.pb'

In [18]:
!ls -l message-classification-model-dir
!ls -l message-classification-model-dir/variables

total 136
-rw-r--r--  1 jinsalex  staff  68351 Jun 12 11:25 saved_model.pb
drwxr-xr-x  4 jinsalex  staff    128 Jun 12 11:25 variables
total 336
-rw-r--r--  1 jinsalex  staff  167432 Jun 12 11:25 variables.data-00000-of-00001
-rw-r--r--  1 jinsalex  staff     496 Jun 12 11:25 variables.index


In [19]:
# Save the model in a tar.gz file
!tar -zcvf message-classification-model.tar.gz -C message-classification-model-dir .

a .
a ./variables
a ./saved_model.pb
a ./variables/variables.data-00000-of-00001
a ./variables/variables.index


In [20]:
!ls -l

total 336
drwxr-xr-x  4 jinsalex  staff     128 Jun 12 11:25 message-classification-model-dir
-rw-r--r--  1 jinsalex  staff  111005 Jun 12 11:25 message-classification-model.tar.gz
-rw-r--r--  1 jinsalex  staff    5781 Jun 12 11:25 sample-customer-messages.csv
-rw-r--r--  1 jinsalex  staff   48664 Jun 11 15:50 tensorflow-import-python.ipynb


## <a id="step1"></a> Step 1: Store the model in your Watson Machine Learning repository

This section of the notebook demonstrates calling the <a href="https://wml-api-pyclient.mybluemix.net/index.html?highlight=store_model#client.Repository.store_model" target="_blank" rel="noopener noreferrer">store_model</a> function

Paste your Watson Machine Learning credentials in the following cell.

See: <a href="https://dataplatform.cloud.ibm.com/docs/content/analyze-data/ml-get-wml-credentials.html" target="_blank" rel="noopener noreferrer">Looking up credentials</a>

In [21]:
# Create a Watson Machine Learning client instance
from watson_machine_learning_client import WatsonMachineLearningAPIClient
wml_credentials = {
    "apikey"    : "value",
    "instance_id" : "instance_id",
    "url"    : "url"
}
client = WatsonMachineLearningAPIClient( wml_credentials )

/Users/jinsalex/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [22]:
# Format 1: Model saved in a directory
#
# Parameters:
# 1. The directory where the model was saved
# 2. Metadata, including a name you choose for the stored model, as well as information about the framework
#
metadata = {
    client.repository.ModelMetaNames.NAME: "TensorFlow model (directory)",
    client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.5"
}
model_details_dir = client.repository.store_model( model="message-classification-model-dir", meta_props=metadata )

In [23]:
# Format 2: Model saved in a directory, and that directory saved in a tar.gz file
#
# Parameters:
# 1. The tar.gz file containing the directory where the model was saved
# 2. Metadata, including a name you choose for the stored model, as well as information about the framework
#
metadata = {
    client.repository.ModelMetaNames.NAME: "TensorFlow model (tar.gz)",
    client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.5"
}
model_details_targz = client.repository.store_model( model="message-classification-model.tar.gz", meta_props=metadata )

## <a id="step2"></a> Step 2: Deploy the stored the model in your Watson Machine Learning service

This section of the notebook demonstrates calling the <a href="https://wml-api-pyclient.mybluemix.net/index.html?highlight=deploy#client.Deployments.create" target="_blank" rel="noopener noreferrer">deployments.create</a> function

In [24]:
# Deploy the stored model as an online web service deployment
model_id_dir = model_details_dir["metadata"]["guid"]
deployment_details_dir = client.deployments.create( artifact_uid=model_id_dir, name="TensorFlow deployment (directory)" )



#######################################################################################

Synchronous deployment creation for uid: '00abd83b-7461-40c6-a7a4-8663dfb3cb96' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='27bc1f41-0694-4377-a4d0-1567c61d76cc'
------------------------------------------------------------------------------------------------




In [25]:
# Test the deployment
model_endpoint_url_dir = client.deployments.get_scoring_url( deployment_details_dir )
payload = { "values" : X_test.tolist() }
client.deployments.score( model_endpoint_url_dir, payload )

{'values': [1, 0, 0, 0, 0, 1, 1, 1, 0, 1]}

## Summary
In this notebook, you imported a TensorFlow model into Watson Machine Learning using the Watson Machine Learning Python client.

### <a id="authors"></a>Authors

**Sarah Packowski** is a member of the IBM Watson Studio Content Design team in Canada.


<hr>
Copyright &copy; IBM Corp. 2019. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>